In [9]:
#유사 데이터 제거 1
    # dialogue_data_0 폴더에 있는 txt데이터의 값들을 유사도 계산해서 제거함
    # threshold=0.6 유사도가 60%만 되어도 제거
    # 제거 후 csv파일로 추출
    # 제작된 원 txt파일 데이터 상태가 달라,txt 파일 종류 마다 다른 코드를 사용함

# TXT 파일에서 대화를 추출하고, '대화 n' 형태의 머리말과 의미적으로 유사한 중복 데이터를 제거한 후 글자수를 계산하여 CSV 파일로 저장한다.
import os
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def process_and_deduplicate_dialogues(input_folder, output_csv, threshold=0.6):
    # input_folder: TXT 파일들이 저장된 폴더 경로를 지정한다.
    # output_csv: 최종적으로 저장될 CSV 파일의 이름을 지정한다.
    # threshold: 유사도 판단 기준값이다. 0.0 ~ 1.0 사이의 값으로, 1에 가까울수록 엄격하다.

    all_dialogues = [] # 모든 대화 데이터를 저장할 리스트를 초기화한다.

    # 1. TXT 파일에서 대화 데이터 추출 및 머리말 제거
    # os.listdir() 함수를 사용하여 지정된 폴더 안의 모든 파일과 폴더 목록을 가져온다.
    for filename in os.listdir(input_folder):
        # 파일 이름이 '.txt'로 끝나는지 확인하여 텍스트 파일만 처리한다.
        if filename.endswith('.txt'):
            # os.path.join() 함수를 사용하여 완전한 파일 경로를 생성한다.
            file_path = os.path.join(input_folder, filename)
            
            # 파일을 'r'(읽기) 모드로 열고 UTF-8 인코딩을 지정하여 유니코드 문자를 처리한다.
            with open(file_path, 'r', encoding='utf-8') as file:
                # 파일의 전체 내용을 하나의 문자열로 읽어온다.
                content = file.read()
                # '---'를 기준으로 문자열을 분리하여 개별 대화 목록을 만든다.
                dialogues = content.split('---')
                
                # '대화 n' 형태의 머리말을 제거하는 정규 표현식 패턴을 컴파일한다.
                header_regex = re.compile(r'^\s*\*{0,2}대화\s+\d+\s*\*{0,2}\s*\n', re.MULTILINE)

                # 각 대화 블록을 순회한다.
                for dialogue in dialogues:
                    # 각 대화 블록의 양 끝 공백을 제거한다.
                    cleaned_dialogue = dialogue.strip()
                    
                    # 미리 컴파일된 정규 표현식으로 머리말을 제거한다.
                    dialogue_without_header = header_regex.sub('', cleaned_dialogue)
                    
                    # 빈 문자열이 아닌 경우에만 리스트에 추가한다.
                    if dialogue_without_header:
                        # # 줄바꿈 문자를 공백으로 대체하여 한 줄로 만드는 코드를 삭제하거나 수정한다.
                        # final_dialogue = ' '.join(dialogue_without_header.splitlines())
                        # # 최종 가공된 대화를 리스트에 추가한다.
                        # all_dialogues.append(final_dialogue)

                        # 수정된 부분: 줄바꿈을 유지하기 위해 별도의 처리를 하지 않고 추가
                        all_dialogues.append(dialogue_without_header)

    # 추출된 대화 데이터를 데이터프레임으로 변환한다.
    if not all_dialogues:
        print("입력 폴더에 대화 데이터가 없습니다.")
        return
        
    dialogues_df = pd.DataFrame(all_dialogues, columns=['dialogue'])

    # 2. 의미적으로 유사한 대화 중복 제거
    vectorizer = TfidfVectorizer(ngram_range=(1, 2))
    
    tfidf_matrix = vectorizer.fit_transform(dialogues_df['dialogue'])
    
    cosine_sim = cosine_similarity(tfidf_matrix)
    
    duplicates_to_remove = set()
    
    for i in range(len(cosine_sim)):
        for j in range(i + 1, len(cosine_sim)):
            if cosine_sim[i][j] > threshold:
                duplicates_to_remove.add(j)

    final_df = dialogues_df.drop(duplicates_to_remove).reset_index(drop=True)
    
    print(f"총 {len(duplicates_to_remove)}개의 중복 데이터가 제거되었습니다.")
    print(f"최종 데이터 개수: {len(final_df)}")
    
    # 3. 글자수 계산 컬럼 추가
    # 'dialogue' 열의 각 문자열에 대해 len() 함수를 적용하여 글자수를 계산하고 새로운 '글자수' 열을 생성한다.
    final_df['글자수'] = final_df['dialogue'].apply(len)
    
    # 4. 중복이 제거되고 글자수가 추가된 데이터프레임을 CSV 파일로 저장
    # quotechar='"'와 quoting=csv.QUOTE_ALL 옵션을 사용하여 줄바꿈 문자를 포함한 데이터가 깨지지 않도록 처리한다.
    final_df.to_csv(output_csv, index=False, encoding='utf-8', quotechar='"', quoting=1)
    print(f"'{output_csv}' 파일이 성공적으로 생성되었습니다.") # 사용자에게 작업 완료 메시지를 출력한다.

# 함수 사용 예시
# 'dialogue_data' 폴더에 TXT 파일들을 넣어주세요.
# 최종 결과물은 'cleaned_dialogues.csv' 파일로 저장됩니다.
input_folder_path = 'dialogue_data_0'
output_csv_path = 'cleaned_dialogues_0.csv'

process_and_deduplicate_dialogues(input_folder_path, output_csv_path)

총 392개의 중복 데이터가 제거되었습니다.
최종 데이터 개수: 393
'cleaned_dialogues_0.csv' 파일이 성공적으로 생성되었습니다.


In [20]:
#유사 데이터 제거 2
    # dialogue_data_1 폴더에 있는 txt데이터의 값들을 유사도 계산해서 제거함
    # threshold=0.6 유사도가 60%만 되어도 제거
    # 제거 후 csv파일로 추출 

# TXT 파일에서 대화를 추출하고, '대화 n' 형태의 머리말과 의미적으로 유사한 중복 데이터를 제거한 후 글자수를 계산하여 CSV 파일로 저장한다.
#       각 대화는 CSV 파일의 셀 내에서 줄 내림 처리된다.

import os  # 운영체제와 상호작용하는 기능을 제공하는 모듈이다.
import pandas as pd  # 데이터 분석 및 조작을 위한 라이브러리이다.
import re  # 정규 표현식(Regular Expression)을 사용하여 문자열을 다루는 모듈이다.
from sklearn.feature_extraction.text import TfidfVectorizer  # 텍스트를 숫자 벡터로 변환하는 데 사용한다.
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도를 계산하는 데 사용한다.

def process_and_deduplicate_dialogues(input_folder, output_csv, threshold=0.6):
    # input_folder: TXT 파일들이 저장된 폴더 경로를 지정한다.
    # output_csv: 최종적으로 저장될 CSV 파일의 이름을 지정한다.
    # threshold: 유사도 판단 기준값이다. 0.0 ~ 1.0 사이의 값으로, 1에 가까울수록 엄격하다.

    all_dialogues = []  # 모든 대화 데이터를 저장할 리스트를 초기화한다.

    # 1. TXT 파일에서 대화 데이터 추출 및 전처리
    # os.listdir() 함수를 사용하여 지정된 폴더 안의 모든 파일과 폴더 목록을 가져온다.
    for filename in os.listdir(input_folder):
        # 파일 이름이 '.txt'로 끝나는지 확인하여 텍스트 파일만 처리한다.
        if filename.endswith('.txt'):
            # os.path.join() 함수를 사용하여 완전한 파일 경로를 생성한다.
            file_path = os.path.join(input_folder, filename)
            
            # 파일을 'r'(읽기) 모드로 열고 UTF-8 인코딩을 지정하여 유니코드 문자를 처리한다.
            with open(file_path, 'r', encoding='utf-8') as file:
                # 파일의 전체 내용을 하나의 문자열로 읽어온다.
                content = file.read()
                
                # <start> 태그를 기준으로 대화를 분리한다.
                # re.split()을 사용하여 <start> 태그를 기준으로 분리하고,
                # 이 과정에서 태그는 제거된다.
                dialogues = re.split(r'<start>', content)
                
                # 각 대화 블록을 순회하며 불필요한 태그와 공백을 제거한다.
                for dialogue in dialogues:
                    # 빈 문자열과 줄바꿈만 있는 경우를 제외하고 처리한다.
                    if dialogue.strip():
                        # <end> 태그와 그 뒤의 모든 공백을 제거한다.
                        cleaned_dialogue = dialogue.split('<end>')[0].strip()
                        
                        # 불필요한 머리말 제거
                        header_regex = re.compile(r'^\s*대화\s+\d+\s*\(턴수:.*?\)\s*\n')
                        cleaned_dialogue = header_regex.sub('', cleaned_dialogue, count=1).strip()
                        
                        # 수정된 부분: 대화 앞뒤의 큰따옴표를 제거한다.
                        if cleaned_dialogue.startswith('"') and cleaned_dialogue.endswith('"'):
                            cleaned_dialogue = cleaned_dialogue[1:-1]
                        
                        # 최종적으로 남은 대화 내용만 리스트에 추가한다.
                        if cleaned_dialogue:
                            all_dialogues.append(cleaned_dialogue)

    # 추출된 대화 데이터를 데이터프레임으로 변환한다.
    if not all_dialogues:
        print("입력 폴더에 대화 데이터가 없습니다.")
        return
        
    dialogues_df = pd.DataFrame(all_dialogues, columns=['dialogue'])
    print(f"총 {len(dialogues_df)}개의 대화 데이터를 추출했습니다.")

    # 2. 의미적으로 유사한 대화 중복 제거
    vectorizer = TfidfVectorizer(ngram_range=(1, 2))
    
    tfidf_matrix = vectorizer.fit_transform(dialogues_df['dialogue'])
    
    cosine_sim = cosine_similarity(tfidf_matrix)
    
    duplicates_to_remove = set()
    
    for i in range(len(cosine_sim)):
        for j in range(i + 1, len(cosine_sim)):
            if cosine_sim[i][j] > threshold:
                duplicates_to_remove.add(j)

    final_df = dialogues_df.drop(duplicates_to_remove).reset_index(drop=True)
    
    print(f"총 {len(duplicates_to_remove)}개의 중복 데이터가 제거되었습니다.")
    print(f"최종 데이터 개수: {len(final_df)}")
    
    # 3. 글자수 계산 컬럼 추가
    # 'dialogue' 열의 각 문자열에 대해 len() 함수를 적용하여 글자수를 계산하고 새로운 '글자수' 열을 생성한다.
    final_df['글자수'] = final_df['dialogue'].apply(len)
    
    # 4. 중복이 제거되고 글자수가 추가된 데이터프레임을 CSV 파일로 저장
    # pandas는 기본적으로 셀 내부에 줄바꿈 문자가 있으면 해당 셀을 따옴표로 묶어 처리한다.
    final_df.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"'{output_csv}' 파일이 성공적으로 생성되었습니다.") # 사용자에게 작업 완료 메시지를 출력한다.

# 함수 사용 예시
# 'dialogue_data_1' 폴더에 TXT 파일들을 넣어주세요.
# 최종 결과물은 'cleaned_dialogues_1.csv' 파일로 저장됩니다.
input_folder_path = 'dialogue_data_1'
output_csv_path = 'cleaned_dialogues_1.csv'

process_and_deduplicate_dialogues(input_folder_path, output_csv_path)

총 566개의 대화 데이터를 추출했습니다.
총 248개의 중복 데이터가 제거되었습니다.
최종 데이터 개수: 318
'cleaned_dialogues_1.csv' 파일이 성공적으로 생성되었습니다.


In [17]:
#유사 데이터 제거 3
    # dialogue_data_2 폴더에 있는 txt데이터의 값들을 유사도 계산해서 제거함
    # threshold=0.6 유사도가 60%만 되어도 제거
    # 제거 후 csv파일로 추출 

# TXT 파일에서 대화를 추출하고, '대화 n' 형태의 머리말과 의미적으로 유사한 중복 데이터를 제거한 후 글자수를 계산하여 CSV 파일로 저장한다.
import os
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def process_and_deduplicate_dialogues(input_folder, output_csv, threshold=0.6):
    # input_folder: TXT 파일들이 저장된 폴더 경로를 지정한다.
    # output_csv: 최종적으로 저장될 CSV 파일의 이름을 지정한다.
    # threshold: 유사도 판단 기준값이다. 0.0 ~ 1.0 사이의 값으로, 1에 가까울수록 엄격하다.

    all_dialogues = [] # 모든 대화 데이터를 저장할 리스트를 초기화한다.

    # 1. TXT 파일에서 대화 데이터 추출 및 전처리
    # os.listdir() 함수를 사용하여 지정된 폴더 안의 모든 파일과 폴더 목록을 가져온다.
    for filename in os.listdir(input_folder):
        # 파일 이름이 '.txt'로 끝나는지 확인하여 텍스트 파일만 처리한다.
        if filename.endswith('.txt'):
            # os.path.join() 함수를 사용하여 완전한 파일 경로를 생성한다.
            file_path = os.path.join(input_folder, filename)
            
            # 파일을 'r'(읽기) 모드로 열고 UTF-8 인코딩을 지정하여 유니코드 문자를 처리한다.
            with open(file_path, 'r', encoding='utf-8') as file:
                # 파일의 전체 내용을 하나의 문자열로 읽어온다.
                content = file.read()
                
                # 대화 헤더와 관련된 모든 메타데이터를 제거한다.
                cleaned_content = re.sub(r'##.*?---', '', content, flags=re.DOTALL)
                
                # 비정상 머릿말과 정상 머릿말 모두를 분리 기준으로 삼는다.
                # '자'와 '字' 모두를 포함하도록 정규 표현식 수정
                # 비정상 머릿말과 정상 머릿말을 모두 처리할 수 있는 패턴.
                # re.split()은 패턴을 기준으로 문자열을 분할하며,
                # 패턴 자체는 결과에 포함시키지 않는다.
                dialogues = re.split(r'\s*### 대화 \d+ \(턴수: \d+, 길이: \d+[자字]\)\n', cleaned_content)
                
                # 각 대화 블록을 순회한다.
                for dialogue in dialogues:
                    # 각 대화 블록의 양 끝 공백을 제거한다.
                    cleaned_dialogue = dialogue.strip()
                    
                    # 빈 문자열이 아닌 경우에만 리스트에 추가한다.
                    if cleaned_dialogue:
                        all_dialogues.append(cleaned_dialogue)

    # 추출된 대화 데이터를 데이터프레임으로 변환한다.
    if not all_dialogues:
        print("입력 폴더에 대화 데이터가 없습니다.")
        return
        
    dialogues_df = pd.DataFrame(all_dialogues, columns=['dialogue'])
    print(f"총 {len(dialogues_df)}개의 대화 데이터를 추출했습니다.")

    # 2. 의미적으로 유사한 대화 중복 제거
    vectorizer = TfidfVectorizer(ngram_range=(1, 2))
    
    tfidf_matrix = vectorizer.fit_transform(dialogues_df['dialogue'])
    
    cosine_sim = cosine_similarity(tfidf_matrix)
    
    duplicates_to_remove = set()
    
    for i in range(len(cosine_sim)):
        for j in range(i + 1, len(cosine_sim)):
            if cosine_sim[i][j] > threshold:
                duplicates_to_remove.add(j)

    final_df = dialogues_df.drop(duplicates_to_remove).reset_index(drop=True)
    
    print(f"총 {len(duplicates_to_remove)}개의 중복 데이터가 제거되었습니다.")
    print(f"최종 데이터 개수: {len(final_df)}")
    
    # 3. 글자수 계산 컬럼 추가
    # 'dialogue' 열의 각 문자열에 대해 len() 함수를 적용하여 글자수를 계산하고 새로운 '글자수' 열을 생성한다.
    final_df['글자수'] = final_df['dialogue'].apply(len)
    
    # 4. 중복이 제거되고 글자수가 추가된 데이터프레임을 CSV 파일로 저장
    final_df.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"'{output_csv}' 파일이 성공적으로 생성되었습니다.") # 사용자에게 작업 완료 메시지를 출력한다.

# 함수 사용 예시
# 'dialogue_data_4' 폴더에 TXT 파일들을 넣어주세요.
# 최종 결과물은 'cleaned_dialogues_4.csv' 파일로 저장됩니다.
input_folder_path = 'dialogue_data_2'
output_csv_path = 'cleaned_dialogues_2.csv'

process_and_deduplicate_dialogues(input_folder_path, output_csv_path)

총 131개의 대화 데이터를 추출했습니다.
총 64개의 중복 데이터가 제거되었습니다.
최종 데이터 개수: 67
'cleaned_dialogues_2.csv' 파일이 성공적으로 생성되었습니다.


In [25]:
#유사 데이터 제거 4
    # dialogue_data_3 폴더에 있는 txt데이터의 값들을 유사도 계산해서 제거함
    # threshold=0.6 유사도가 60%만 되어도 제거
    # 제거 후 csv파일로 추출 

# TXT 파일에서 대화를 추출하고, '대화 n' 형태의 머리말과 의미적으로 유사한 중복 데이터를 제거한 후 글자수를 계산하여 CSV 파일로 저장한다.
#       각 대화는 CSV 파일의 셀 내에서 줄 내림 처리된다.

import os  # 운영체제와 상호작용하는 기능을 제공하는 모듈이다.
import pandas as pd  # 데이터 분석 및 조작을 위한 라이브러리이다.
import re  # 정규 표현식(Regular Expression)을 사용하여 문자열을 다루는 모듈이다.
from sklearn.feature_extraction.text import TfidfVectorizer  # 텍스트를 숫자 벡터로 변환하는 데 사용한다.
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도를 계산하는 데 사용한다.

def process_and_deduplicate_dialogues(input_folder, output_csv, threshold=0.1):
    # input_folder: TXT 파일들이 저장된 폴더 경로를 지정한다.
    # output_csv: 최종적으로 저장될 CSV 파일의 이름을 지정한다.
    # threshold: 유사도 판단 기준값이다. 0.0 ~ 1.0 사이의 값으로, 1에 가까울수록 엄격하다.

    all_dialogues = []  # 모든 대화 데이터를 저장할 리스트를 초기화한다.

    # 1. TXT 파일에서 대화 데이터 추출 및 전처리
    # os.listdir() 함수를 사용하여 지정된 폴더 안의 모든 파일과 폴더 목록을 가져온다.
    for filename in os.listdir(input_folder):
        # 파일 이름이 '.txt'로 끝나는지 확인하여 텍스트 파일만 처리한다.
        if filename.endswith('.txt'):
            # os.path.join() 함수를 사용하여 완전한 파일 경로를 생성한다.
            file_path = os.path.join(input_folder, filename)
            
            # 파일을 'r'(읽기) 모드로 열고 UTF-8 인코딩을 지정하여 유니코드 문자를 처리한다.
            with open(file_path, 'r', encoding='utf-8') as file:
                # 파일의 전체 내용을 하나의 문자열로 읽어온다.
                content = file.read()
                
                # <start> 태그를 기준으로 대화를 분리한다.
                # re.split()을 사용하여 <start> 태그를 기준으로 분리하고,
                # 이 과정에서 태그는 제거된다.
                dialogues = re.split(r'<start>', content)
                
                # 각 대화 블록을 순회하며 불필요한 태그와 공백을 제거한다.
                for dialogue in dialogues:
                    # 빈 문자열과 줄바꿈만 있는 경우를 제외하고 처리한다.
                    if dialogue.strip():
                        # <end> 태그와 그 뒤의 모든 공백을 제거한다.
                        cleaned_dialogue = dialogue.split('<end>')[0].strip()
                        
                        # 불필요한 머리말 제거
                        header_regex = re.compile(r'^\s*대화\s+\d+\s*\(턴수:.*?\)\s*\n')
                        cleaned_dialogue = header_regex.sub('', cleaned_dialogue, count=1).strip()
                        
                        # 수정된 부분: 대화 앞뒤의 큰따옴표를 제거한다.
                        if cleaned_dialogue.startswith('"') and cleaned_dialogue.endswith('"'):
                            cleaned_dialogue = cleaned_dialogue[1:-1]
                        
                        # 최종적으로 남은 대화 내용만 리스트에 추가한다.
                        if cleaned_dialogue:
                            all_dialogues.append(cleaned_dialogue)

    # 추출된 대화 데이터를 데이터프레임으로 변환한다.
    if not all_dialogues:
        print("입력 폴더에 대화 데이터가 없습니다.")
        return
        
    dialogues_df = pd.DataFrame(all_dialogues, columns=['dialogue'])
    print(f"총 {len(dialogues_df)}개의 대화 데이터를 추출했습니다.")

    # 2. 의미적으로 유사한 대화 중복 제거
    vectorizer = TfidfVectorizer(ngram_range=(1, 2))
    
    tfidf_matrix = vectorizer.fit_transform(dialogues_df['dialogue'])
    
    cosine_sim = cosine_similarity(tfidf_matrix)
    
    duplicates_to_remove = set()
    
    for i in range(len(cosine_sim)):
        for j in range(i + 1, len(cosine_sim)):
            if cosine_sim[i][j] > threshold:
                duplicates_to_remove.add(j)

    final_df = dialogues_df.drop(duplicates_to_remove).reset_index(drop=True)
    
    print(f"총 {len(duplicates_to_remove)}개의 중복 데이터가 제거되었습니다.")
    print(f"최종 데이터 개수: {len(final_df)}")
    
    # 3. 글자수 계산 컬럼 추가
    # 'dialogue' 열의 각 문자열에 대해 len() 함수를 적용하여 글자수를 계산하고 새로운 '글자수' 열을 생성한다.
    final_df['글자수'] = final_df['dialogue'].apply(len)
    
    # 4. 중복이 제거되고 글자수가 추가된 데이터프레임을 CSV 파일로 저장
    # pandas는 기본적으로 셀 내부에 줄바꿈 문자가 있으면 해당 셀을 따옴표로 묶어 처리한다.
    final_df.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"'{output_csv}' 파일이 성공적으로 생성되었습니다.") # 사용자에게 작업 완료 메시지를 출력한다.

# 함수 사용 예시
# 'dialogue_data_1' 폴더에 TXT 파일들을 넣어주세요.
# 최종 결과물은 'cleaned_dialogues_1.csv' 파일로 저장됩니다.
input_folder_path = 'dialogue_data_3'
output_csv_path = 'cleaned_dialogues_3.csv'

process_and_deduplicate_dialogues(input_folder_path, output_csv_path)

총 2683개의 대화 데이터를 추출했습니다.
총 2221개의 중복 데이터가 제거되었습니다.
최종 데이터 개수: 462
'cleaned_dialogues_3.csv' 파일이 성공적으로 생성되었습니다.


In [26]:
# 데이터 셔플
    # 최종 만들어진 데이터를 모델에 넣기전에 셔플

import pandas as pd

# 1. CSV 파일 불러오기
# 'train_mix_1.csv' 파일에 'class'와 'conversation' 칼럼이 있다고 가정
file_path = "train_mix_2.csv"
try:
    df = pd.read_csv(file_path)
    print("✅ 원본 데이터프레임의 상위 5개 행:")
    print(df.head())
    print("-" * 30)

    # 2. 데이터프레임 셔플 (섞기)
    # frac=1은 전체 데이터를 셔플하겠다는 의미입니다.
    # random_state를 설정하면 매번 동일한 결과로 셔플되어 재현성이 보장됩니다.
    shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    print("✅ 셔플된 데이터프레임의 상위 5개 행:")
    print(shuffled_df.head())

    # 3. 셔플된 데이터를 새로운 CSV 파일로 저장 (선택 사항)
    shuffled_df.to_csv("train_mix_2_s.csv", index=False, encoding="utf-8-sig")
    print("✅ 셔플된 데이터가 'shuffled_train_mix_1.csv' 파일로 저장되었습니다.")

except FileNotFoundError:
    print(f"오류: {file_path} 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")

✅ 원본 데이터프레임의 상위 5개 행:
   class                                       conversation
0      0  지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1      0  길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2      3  너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3      1  어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4      1  저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
------------------------------
✅ 셔플된 데이터프레임의 상위 5개 행:
   class                                       conversation
0      1  친구야 6만원만 빌려줘라\n어?.나 이거 학교책사야되는데\n아 딱 하루만 쓰고 갚을...
1      0  나 분명히 말했다\n왜 이러세요 대화로해요\n대화하자는 놈이 계속 그래?\n신고할거...
2      0  너 아들이 내 딸 학창시절을 지옥으로 만들었다 마지막이라도 사진 잘봐라\n다 제가 ...
3      1  노랑아 나 담주 생일인거 알지?\n네? 아.네 미리 축하드려요\n뭐야 설마 말로 때...
4      2  어 최간호사 오늘 혹시 출근해줄 수 있어?\n아. 저 오늘 오프여서 이미 다른 지역...
✅ 셔플된 데이터가 'shuffled_train_mix_1.csv' 파일로 저장되었습니다.
